/data/CLAM

### KIRC, KIRP, GBMLGG, PAAD

gdc-client download -m /home/wbjeong/cancer/WSI/gdc_manifest_PAAD_diagnostic_slides.txt -d /data/CLAM/PAAD

/home/wbjeong/cancer/WSI/gdc_manifest_LUAD_diagnostic_slide.txt

In [ ]:
cp -r /home/wbjeong/cancer/JJM/WSI/Data2/BLCA/* /data/CLAM/BLCA
cp -r /home/wbjeong/cancer/JJM/WSI/Data2/KIRC/* /data/CLAM/KIRC

In [4]:
import os
import shutil

name = 'SKCM'

dir_path = '/data/CLAM/' + name + '/'
count = 0
for subdir in os.listdir(dir_path):
    subdir_path = os.path.join(dir_path, subdir)

    if os.path.isdir(subdir_path):
        for file in os.listdir(subdir_path):
            if file.endswith('.svs'):
                file_path = os.path.join(subdir_path, file)
                shutil.move(file_path, dir_path)
                count += 1

print(f"Total .svs files moved: {count}")

Total .svs files moved: 374


In [5]:
dir_path = '/data/CLAM/' + name + '/'

for item in os.listdir(dir_path):
    item_path = os.path.join(dir_path, item)
    
    dir_count = 0
    file_count = 0
    
    if os.path.isdir(item_path):
        shutil.rmtree(item_path)
        dir_count += 1
        
    else:
        file_count += 1
        
print(f"Removed directory: {dir_count}")    
print(f"Skipped file: {file_count}")

Removed directory: 0
Skipped file: 1


In [6]:
name = 'STAD'

dirs_path = '/data/CLAM/'
output_dir = '/home/wbjeong/cancer/WSI/'

### output_dir에 있는 gdc_manifest_PAAD_diagnostic_slides.txt 의 filename 중 dir_path에 존재하지 않는 filename이 있다면 모두 출력 및 개수 출력

with open('/home/wbjeong/cancer/WSI/gdc_manifest_' + name + '_diagnostic_slides.txt', 'r') as f:
    filenames = f.readlines()
    filenames = [filename.split('\t')[1] for filename in filenames]
    filenames = [filename for filename in filenames if not 'filename' in filename and not 'slide_id' in filename and not 'file_id' in filename]
    print(len(filenames))
    
    txt_count = 0
    file_count = 0
    
    for filename in filenames:
        if not os.path.exists(os.path.join(dirs_path + name, filename)):
            # print(filename)
            txt_count += 1

    for filen in os.listdir(dirs_path + name):
        if not filen in filenames:
            # print(filen)
            file_count += 1

print(f"File not downloaded: {txt_count}")
print(f"File not in txt: {file_count}")

374
File not downloaded: 0
File not in txt: 0


In [4]:
import os
import pandas as pd

dir = '/home/wbjeong/cancer/PORPOISE/datasets_csv_mutsig/'
input_dir = '/home/wbjeong/cancer/WSI/gdc/'
output_dir = '/home/wbjeong/cancer/WSI/'
cancer_type = ['BLCA', 'BRCA', 'COADREAD', 'GBMLGG', 'HNSC', 'KIRC', 'KIRP', 'LIHC', 'LUAD', 'LUSC', 'PAAD', 'SKCM', 'STAD', 'UCEC']

### 각각의 cancer_type에 대해, dir에서 tcga_ + 소문자로 바꾼 각각의 cancer_type으로 시작하는 파일을 ref로 읽어들인다.
### 동시에, input_dir에서 gdc_manifest_ + 소문자로 바꾼 각각의 cancer_type으로 시작하는 파일을 alt로 읽어들인다.
### 이때 alt에 filname 중 ref의 slide_id 에서 .svs를 제외한 문자열을 포함하는 filename만 남기고 나머지는 제거한다.
### alt를 output_dir에 gdc_manifest_ + cancer_type (대문자) + _diagnostic_slides.txt로 저장한다.

for cancer in cancer_type:
    
    ### ref는 zip 파일이므로, compression='zip'으로 읽어들인다.
    
    ref = pd.read_csv(dir + 'tcga_' + cancer.lower() + '_all_clean.csv.zip', compression='zip', low_memory=False)
    alt = pd.read_csv(input_dir + 'gdc_manifest_' + cancer.lower() + '.txt', sep='\t', low_memory=False)
    # print(len(alt))
    alt = alt[alt['filename'].str.contains('|'.join(ref['slide_id'].str.replace('.svs', '')))]
    patients = len(ref['case_id'].unique())
    
    print(cancer, len(alt), patients)
    
    # alt.to_csv(output_dir + 'gdc_manifest_' + cancer + '_diagnostic_slides.txt', sep='\t', index=False)
    # print(cancer + ' done')

/home/wbjeong/.conda/envs/py37-cuda10-cudnn7/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: The default value of regex will change from True to False in a future version.


BLCA 437 373
BRCA 1022 956
COADREAD 344 340
GBMLGG 822 569
HNSC 437 416
KIRC 350 345
KIRP 284 260
LIHC 347 333
LUAD 515 453
LUSC 484 450
PAAD 180 166
SKCM 268 230
STAD 374 349
UCEC 538 480


### 위 개수와 동일한지 확인할 것

ls -l | sort | uniq | wc -l

### 중간에 로딩에러로 끊긴 파일 개수 및 이름 확인

find . -type f -name '*partial*' | wc -l

find . -type f -name '*partial*' -exec dirname {} \; | sort | uniq

### 만약 확인된 이름이 다음과 같다면, 수동적으로 해당 파일 이름을 집어넣어서 다운로드 할 것

8bad070b-5d83-4148-b773-c6f579c8bb3f

gdc-client download 8bad070b-5d83-4148-b773-c6f579c8bb3f -d /data/CLAM/KIRP


### 현재 사용 screen 근황

amil : extracting (BRCA)
filemove : patching (STAD)
cancer : patching (LIHC)
snn : extracting (PAAD)



In [8]:
import subprocess

cancer_types = [
    'BLCA', 'BRCA', 'COADREAD', 'GBMLGG', 'HNSC', 
    'KIRC', 'KIRP', 'LIHC', 'LUAD', 'LUSC', 
    'PAAD', 'SKCM', 'STAD', 'UCEC'
]

base_path = '/data/CLAM/'

# Function to execute a shell command and return the output
def run_command(command):
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout.strip()

sum_du = 0
sum_ls_wc = 0

for cancer in cancer_types:
    path = base_path + cancer + '/'
    
    # Run 'du -h' command
    du_command = f'du -sh {path}'
    du_result = run_command(du_command)
    ### 맨 뒤 G를 제거하고 int로 변환
    sum_du += int(du_result.split()[0][:-1])
    
    
    # Run 'ls -l | wc -l' command
    ls_wc_command = f'ls -l {path} | wc -l'
    ls_wc_result = run_command(ls_wc_command)
    ls_wc_result = int(ls_wc_result) - 1
    sum_ls_wc += ls_wc_result
    
    print(f'Cancer Type: {cancer}', du_result, ls_wc_result)
    
print(f'Total Disk Usage: {sum_du} , Total Number of Files: {sum_ls_wc}')

Cancer Type: BLCA 665G	/data/CLAM/BLCA/ 457
Cancer Type: BRCA 983G	/data/CLAM/BRCA/ 1022
Cancer Type: COADREAD 221G	/data/CLAM/COADREAD/ 344
Cancer Type: GBMLGG 880G	/data/CLAM/GBMLGG/ 822
Cancer Type: HNSC 398G	/data/CLAM/HNSC/ 437
Cancer Type: KIRC 438G	/data/CLAM/KIRC/ 515
Cancer Type: KIRP 268G	/data/CLAM/KIRP/ 284
Cancer Type: LIHC 424G	/data/CLAM/LIHC/ 347
Cancer Type: LUAD 387G	/data/CLAM/LUAD/ 541
Cancer Type: LUSC 367G	/data/CLAM/LUSC/ 484
Cancer Type: PAAD 201G	/data/CLAM/PAAD/ 180
Cancer Type: SKCM 295G	/data/CLAM/SKCM/ 268
Cancer Type: STAD 302G	/data/CLAM/STAD/ 374
Cancer Type: UCEC 763G	/data/CLAM/UCEC/ 538
Total Disk Usage: 6592 , Total Number of Files: 6613


In [1]:
import numpy as np
import pandas as pd
import os

### Load the directory of the WSI data
dir = '/home/wbjeong/cancer/PORPOISE/datasets_csv_mutsig/'

### check names of the files in the directory
files = os.listdir(dir)
print(files)

['tcga_blca_all_clean.csv.zip', 'tcga_brca_all_clean.csv.zip', 'tcga_coadread_all_clean.csv.zip', 'tcga_hnsc_all_clean.csv.zip', 'tcga_kirc_all_clean.csv.zip', 'tcga_kirp_all_clean.csv.zip', 'tcga_lihc_all_clean.csv.zip', 'tcga_luad_all_clean.csv.zip', 'tcga_lusc_all_clean.csv.zip', 'tcga_paad_all_clean.csv.zip', 'tcga_skcm_all_clean.csv.zip', 'tcga_stad_all_clean.csv.zip', 'tcga_ucec_all_clean.csv.zip', 'tcga_gbmlgg_all_clean.csv.zip']


In [ ]:
# 'tcga_blca_all_clean.csv.zip' 라는 파일의 head=5

### Load the data
data = pd.read_csv(dir + 'tcga_luad_all_clean.csv.zip', compression='zip')
print(data.head(5))

### slide_id만 추출
slide_id = data['slide_id']
### 맨 뒤의 .svs를 제거
slide_id = slide_id.str.replace('.svs', '')
print(len(slide_id))

move_dir = '/data/CLAM/LUAD_RESULT/FEATURE/h5_files'
move2_dir = '/data/CLAM/LUAD_RESULT/FEATURE/pt_files'

into_dir = '/data/CLAM/LUAD_RESULT/AMIL/h5_files'
into2_dir = '/data/CLAM/LUAD_RESULT/AMIL/pt_files'

# into_dir과 into2_dir 디렉토리가 없다면 생성하고, slide_id와 부분일치하는 파일을 move_dir, move2_dir에서 into_dir, into2_dir로 이동시킨다.
if not os.path.exists(into_dir):
    os.makedirs(into_dir)
    
if not os.path.exists(into2_dir):
    os.makedirs(into2_dir)
    
for i in range(len(slide_id)):
    file_name = slide_id[i]
    print(file_name)
    if os.path.exists(move_dir + '/' + file_name + '.h5'):
        os.rename(move_dir + '/' + file_name + '.h5', into_dir + '/' + file_name + '.h5')
    if os.path.exists(move2_dir + '/' + file_name + '.pt'):
        os.rename(move2_dir + '/' + file_name + '.pt', into2_dir + '/' + file_name + '.pt')

print('done')


In [ ]:
move_final = '/home/wbjeong/cancer/PORPOISE/path/to/data_root_dir/tcga_lung_20x_features/pt_files'
    
### into2_dir = '/data/CLAM/LUAD_RESULT/AMIL/pt_files'에 존재하는 파일을 move_final 모두 복사
import shutil
import os

count = 0
for filename in os.listdir(into2_dir):
    source_file = os.path.join(into2_dir, filename)
    destination_file = os.path.join(move_final, filename)
    
    if os.path.isfile(source_file):
        shutil.copy2(source_file, destination_file)
        count += 1
        
print(count)

In [ ]:
dd = 'TCGA-86-8076-01Z-00-DX1.e7378b2f-e20e-4d2f-a86c-3a8ead08a385'

### dd가 slide_id에 있는지 확인

if dd in slide_id:
    print('yes')
else:
    print('no')
    
## python main.py --which_splits 5foldcv --split_dir tcga_luad --mode path --model_type porpoise_amil

In [ ]:
files = os.listdir(dir)
files = [file for file in files if file.endswith('.zip')]

out_dir = '/data/Cancer/'

file_count_df = pd.DataFrame(columns=['slide_id', 'count'])

for file in files:
    df = pd.read_csv(dir + file, compression='zip', low_memory=False, dtype={'slide_id': str})
    slide_id = file.split('_')[1]
    name = 'tcga_' + slide_id + '_name.txt'
    # print(name)
    # print(df.shape[0])
    file_count_df = file_count_df.append({'slide_id': slide_id, 'count': df.shape[0]}, ignore_index=True)
    
    # df.to_csv(out_dir + name, sep='\t', index=False)

file_count_df = file_count_df.T
file_count_df.columns = file_count_df.iloc[0]
file_count_df['total'] = file_count_df.sum(axis=1) 
file_count_df = file_count_df.drop(file_count_df.index[0])
print(file_count_df)


In [ ]:
input_dir = '/home/wbjeong/cancer/WSI/gdc/'

### 해당 디렉토리의 모든 txt 파일을 하나로 합치고, 이때 header는 맨 처음에만 존재하도록 한다.
all_tcga = pd.DataFrame()

for file in os.listdir(input_dir):
    if file.endswith('.txt'):
        df = pd.read_csv(input_dir + file, sep='\t', low_memory=False)
        all_tcga = all_tcga.append(df, ignore_index=True)

print(all_tcga.shape)

exists = pd.DataFrame(columns=['slide_id', 'gdc_count'])

for file in files:
    df = pd.read_csv(dir + file, compression='zip', low_memory=False, dtype={'slide_id': str})
    slide_id = file.split('_')[1]
    
    count = 0
    for i in range(df.shape[0]):
        id = df.iloc[i]['slide_id']
        # id = id.split('.')[0] + '.' + id.split('.')[1].lower() + '.' + id.split('.')[2]
        
        if id in all_tcga['filename'].values:
            count += 1
    
    exists = exists.append({'slide_id': slide_id, 'gdc_count': count}, ignore_index=True)
    
print(exists)


In [ ]:
dirr = '/data/CLAM/'

for cancer in cancer_type:
    os.mkdir(dirr + cancer)

In [ ]:
data = pd.read_csv(dir + 'tcga_gbmlgg_all_clean.csv.zip', compression='zip')

# all_tcga의 filename중 data에 있는 slide_id와 일치하는 것의 개수

count = 0
for i in range(data.shape[0]):
    id = data.iloc[i]['slide_id']
    if id in all_tcga['filename'].values:
        count += 1

print(count)

data_trimed = pd.DataFrame(columns=data.columns)
for i in range(data.shape[0]):
    id = data.iloc[i]['slide_id']
    if id in all_tcga['filename'].values:
        data_trimed = data_trimed.append(data.iloc[i], ignore_index=True)

print(len(data_trimed))

### data_trimed를 csv로 저장
data_trimed.to_csv(dir + 'tcga_new_gbmlgg_all_clean_trimed.csv', index=False)

In [ ]:
dir2 = '/home/wbjeong/cancer/WSI/'

all_tcga = pd.DataFrame()

for file in os.listdir(input_dir):
    if file.endswith('.txt'):
        df = pd.read_csv(input_dir + file, sep='\t', low_memory=False)
        all_tcga = all_tcga.append(df, ignore_index=True)

total_id_list = []

for file in files:
    df = pd.read_csv(dir + file, compression='zip', low_memory=False, dtype={'slide_id': str})
    slide_id = file.split('_')[1]

    for i in range(df.shape[0]):
        id = df.iloc[i]['slide_id']
        total_id_list.append(id)
        
print(len(total_id_list))

zip tcga_gbmlgg_all_clean.csv.zip tcga_gbmlgg_all_clean.csv